In [ ]:
import numpy as np
import csv
import pandas as pd

## Load data

In [ ]:
d_x_test_eeg1 = pd.read_csv("test_eeg1.csv") 
d_x_test_eeg2 = pd.read_csv("test_eeg2.csv") 
d_x_test_emg = pd.read_csv("test_emg.csv") 
d_x_train_eeg1 = pd.read_csv("train_eeg1.csv")
d_x_train_eeg2 = pd.read_csv("train_eeg2.csv") 
d_x_train_emg = pd.read_csv("train_emg.csv") 
d_y_train = pd.read_csv("train_labels.csv")


x_test_eeg1 = d_x_test_eeg1.values[:,1:]
x_test_eeg2 = d_x_test_eeg2.values[:,1:]
x_test_emg = d_x_test_emg.values[:,1:]
x_train_eeg1 = d_x_train_eeg1.values[:,1:]
x_train_eeg2 = d_x_train_eeg2.values[:,1:]
x_train_emg = d_x_train_emg.values[:,1:]
y_train = d_y_train['y'].values

## Check data

In [ ]:
print('Check training data.')
print(d_x_train_eeg1.head())
print(d_x_train_eeg1.shape)
print('There are %d NAN values.'%np.sum(np.isnan(x_train_eeg1)))
print(d_x_train_eeg2.head())
print(d_x_train_eeg2.shape)
print('There are %d NAN values.'%np.sum(np.isnan(x_train_eeg2)))
print(d_x_train_emg.head())
print(d_x_train_emg.shape)
print('There are %d NAN values.'%np.sum(np.isnan(x_train_emg)))

In [ ]:
y_size = d_y_train.groupby('y').size()
y1_size = d_y_train.iloc[0:21599,].groupby('y').size()
y2_size = d_y_train.iloc[21600:43199,].groupby('y').size()
y3_size = d_y_train.iloc[43200:64799,].groupby('y').size()
print("y size:",y_size,
      "\n y1 size: ",y1_size,
     "\n y2 size: ",y2_size,
     "\n y3 size: ",y3_size)

## Transform data

In [ ]:
x1_train_eeg1=x_train_eeg1[0:21600,]
x2_train_eeg1=x_train_eeg1[21600:43200,]
x3_train_eeg1=x_train_eeg1[43200:64800,]
x1_test_eeg1=x_test_eeg1[0:21600,]
x2_test_eeg1=x_test_eeg1[21600:43200,]

x1_train_eeg2=x_train_eeg2[0:21600,]
x2_train_eeg2=x_train_eeg2[21600:43200,]
x3_train_eeg2=x_train_eeg2[43200:64800,]
x1_test_eeg2=x_test_eeg2[0:21600,]
x2_test_eeg2=x_test_eeg2[21600:43200,]

x1_train_emg=x_train_emg[0:21600,]
x2_train_emg=x_train_emg[21600:43200,]
x3_train_emg=x_train_emg[43200:64800,]
x1_test_emg=x_test_emg[0:21600,]
x2_test_emg=x_test_emg[21600:43200,]

In [ ]:
x_1=np.zeros((3,21600,512))
x_1[0]=x1_train_eeg1
x_1[1]=x1_train_eeg2
x_1[2]=x1_train_emg

x_2=np.zeros((3,21600,512))
x_2[0]=x2_train_eeg1
x_2[1]=x2_train_eeg2
x_2[2]=x2_train_emg

x_3=np.zeros((3,21600,512))
x_3[0]=x3_train_eeg1
x_3[1]=x3_train_eeg2
x_3[2]=x3_train_emg

x_1_test=np.zeros((3,21600,512))
x_1_test[0]=x1_test_eeg1
x_1_test[1]=x1_test_eeg2
x_1_test[2]=x1_test_emg

x_2_test=np.zeros((3,21600,512))
x_2_test[0]=x2_test_eeg1
x_2_test[1]=x2_test_eeg2
x_2_test[2]=x2_test_emg

In [ ]:
# print(x1_train_eeg1.shape)

## Extract features

In [ ]:
!pip install scipy

In [ ]:
import numpy as np
from scipy import integrate
from biosppy.signals import eeg
from scipy.signal import find_peaks,peak_prominences,peak_widths,periodogram
from scipy.stats import kurtosis,skew
from sklearn.decomposition import PCA

def StatFeature(arrays):
    mean = np.mean(arrays)
    std = np.std(arrays)
    maxv = np.max(arrays)
    minv = np.min(arrays)
    return [mean,std,maxv, minv]


def findLFHF(psd, w):
    VLFpsd = VLFw = LFpsd = LFw = HFpsd = HFw = np.empty(0)
    m = w.shape[0]

    for i in range(0, m):
        if w[i] <= 0.05:
            VLFpsd = np.append(VLFpsd, psd[i])
            VLFw = np.append(VLFw, w[i])
        if w[i] > 0.05 and w[i] <= 0.15:
            LFpsd = np.append(LFpsd, psd[i])
            LFw = np.append(LFw, w[i])
        if w[i] > 0.15 and w[i] <= 0.4:
            HFpsd = np.append(HFpsd, psd[i])
            HFw = np.append(HFw, w[i])

    LF = integrate.trapz(LFw, LFpsd) / (integrate.trapz(w, psd) - integrate.trapz(VLFw, VLFpsd))
    HF = integrate.trapz(HFw, HFpsd) / (integrate.trapz(w, psd) - integrate.trapz(VLFw, VLFpsd))
    LFHFratio = LF / HF
    inter = LF / (LF + HF)
    if HFpsd.size:
        [maxHFD, maxIndex] = max((v, i) for i, v in enumerate(HFpsd))
        FreqmaxP = HFw[maxIndex]
    else:
        maxHFD = 0
        FreqmaxP = 0
    return (LF, HF, FreqmaxP, maxHFD, LFHFratio, inter)


def EMGFeatures(raw_signal, fs=128):
    # Statistical Features
    [_,std,maxv,minv] = StatFeature(raw_signal)
    
    # Power Spectrum
    w = np.hamming(len(raw_signal))       # using Hamming window function
    w, psd = periodogram(raw_signal, window=w, detrend=False)
    _, _, _, maxHFD, _, _ = findLFHF(psd, w)
    
    # Time Series
    kurt = kurtosis(raw_signal)
    sk = skew(raw_signal)
        
    # Peak Features
    [peaks,_] = find_peaks(raw_signal)
    pprom = peak_prominences(raw_signal,peaks)[0]
    contour_heights = raw_signal[peaks] - pprom
    pwid = peak_widths(raw_signal,peaks,rel_height=0.4)[0]
    [ppmean,ppstd,_,ppmin] = StatFeature(pprom)
    [pwmean,pwstd,pwmax,pwmin] = StatFeature(pwid)
 
    return np.array([std,maxv,minv,maxHFD, kurt,sk,ppmean,ppstd,ppmin,pwmean,pwstd,pwmax,pwmin])


def EEGFeatures(raw_signal,signal_1,signal_2, fs=128):
    bio_signal = np.transpose(raw_signal)
        
    # Statistical Features
    [_,std1,maxv1,minv1] = StatFeature(signal_1)
    [_,std2,maxv2,minv2] = StatFeature(signal_2)    
        
    # Power Features
    [_, theta, alpha_low,alpha_high,beta, gamma]= eeg.get_power_features(signal=bio_signal, sampling_rate=fs)
    # seems not good because it use smaller windows to calculate the power
    # don't know how this function is implemented in details
        
    [theta1_mean,theta1_std,theta1_max,theta1_min] = StatFeature(theta[:,0])
    [theta2_mean,theta2_std,theta2_max,theta2_min] = StatFeature(theta[:,1])
    [alpha_low1_mean,alpha_low1_std,alpha_low1_max,alpha_low1_min] = StatFeature(alpha_low[:,0])
    [alpha_low2_mean,alpha_low2_std,alpha_low2_max,alpha_low2_min] = StatFeature(alpha_low[:,1])
    [alpha_high1_mean,alpha_high1_std,alpha_high1_max,alpha_high1_min] = StatFeature(alpha_high[:,0])
    [alpha_high2_mean,alpha_high2_std,alpha_high2_max,alpha_high2_min] = StatFeature(alpha_high[:,1])
    [beta1_mean,beta1_std,beta1_max,beta1_min] = StatFeature(beta[:,0])
    [beta2_mean,beta2_std,beta2_max,beta2_min] = StatFeature(beta[:,1])
    [gamma1_mean,gamma1_std,gamma1_max,gamma1_min] = StatFeature(gamma[:,0])
    [gamma2_mean,gamma2_std,gamma2_max,gamma2_min] = StatFeature(gamma[:,1])
        
        
    # Power Spectrum
    w = np.hamming(len(signal_1))
    w, psd = periodogram(signal_1, window=w, detrend=False)
    _, _, FreqmaxP1, _, _, _ = findLFHF(psd, w)
    w = np.hamming(len(signal_2))
    w, psd = periodogram(signal_2, window=w, detrend=False)
    _, _, FreqmaxP2, _, _, _ = findLFHF(psd, w)
        
    # Time Series
    kurt1 = kurtosis(signal_1)
    skew1 = skew(signal_1)
    kurt2 = kurtosis(signal_2)
    skew2 = skew(signal_2)
        
    # Peak Features
    [peaks1,_] = find_peaks(signal_1)
    pprom1 = peak_prominences(signal_1,peaks1)[0]
    contour_heights1 = signal_1[peaks1] - pprom1
    pwid1 = peak_widths(signal_1,peaks1,rel_height=0.4)[0]
    [ppmean1,ppstd1,_,ppmin1] = StatFeature(pprom1)
    [pwmean1,pwstd1,pwmax1,pwmin1] = StatFeature(pwid1)
        
    [peaks2,_] = find_peaks(signal_2)
    pprom2 = peak_prominences(signal_2,peaks2)[0]
    contour_heights2 = signal_2[peaks2] - pprom2
    pwid2 = peak_widths(signal_2,peaks2,rel_height=0.4)[0]
    [ppmean2,ppstd2,_,ppmin2] = StatFeature(pprom2)
    [pwmean2,pwstd2,pwmax2,pwmin2] = StatFeature(pwid2)
 
    return np.array([theta1_mean,theta1_std,theta1_max,theta1_min, theta2_mean,theta2_std,theta2_max,theta2_min, alpha_low1_mean,alpha_low1_std,alpha_low1_max,alpha_low1_min,alpha_low2_mean,alpha_low2_std,alpha_low2_max,alpha_low2_min,alpha_high1_mean,alpha_high1_std,alpha_high1_max,alpha_high1_min,alpha_high2_mean,alpha_high2_std,alpha_high2_max,alpha_high2_min,beta1_mean,beta1_std,beta1_max,beta1_min,beta2_mean,beta2_std,beta2_max,beta2_min,gamma1_mean,gamma1_std,gamma1_max,gamma1_min,gamma2_mean,gamma2_std,gamma2_max,gamma2_min,FreqmaxP1,kurt1,skew1,ppmean1,ppstd1,ppmin1,pwmean1,pwstd1,pwmax1,pwmin1,FreqmaxP2,kurt2,skew2,ppmean2,ppstd2,ppmin2,pwmean2,pwstd2,pwmax2,pwmin2,std1,maxv1,minv1,std2,maxv2,minv2])


def ExtractFeatures(eeg1,eeg2,emg, fs=128):
    raw_signal = np.concatenate(([eeg1],[eeg2]))
    feature1 = EEGFeatures(raw_signal,eeg1,eeg2,fs=fs)
    feature2 = EMGFeatures(emg)
    return np.concatenate((feature1, feature2))

In [ ]:
w, psd = periodogram(np.concatenate(([x1_train_eeg1[0]],[x1_train_eeg2[0]])), detrend=False)
np.concatenate((x1_train_eeg1[0],x1_train_eeg2[0]))

In [ ]:
# features according to https://www.sciencedirect.com/science/article/pii/S0010482512001588

def AddExtractFeatures(eeg1,eeg2,emg,fs=128):
    raw_signal = np.concatenate((eeg1,eeg2))
    w = np.hamming(raw_signal.shape[0])       # using Hamming window function
    w, psd = periodogram(raw_signal, window=w, detrend=False)
    betapsd = betaw = alphapsd = alphaw = deltapsd = deltaw = thetapsd = thetaw = gammapsd = gammaw = np.empty(0)
    m = w.shape[0]
    for i in range(0, m):
        if w[i]>0.5 and w[i] <= 4:
            deltapsd = np.append(deltapsd, psd[i])
            deltaw = np.append(deltaw, w[i])
        if w[i] > 4 and w[i] <= 8:
            thetapsd = np.append(thetapsd, psd[i])
            thetaw = np.append(thetaw, w[i])
        if w[i] > 8 and w[i] <= 12:
            alphapsd = np.append(alphapsd, psd[i])
            alphaw = np.append(alphaw, w[i])
        if w[i] > 13 and w[i] <= 30:
            betapsd = np.append(betapsd, psd[i])
            betaw = np.append(betaw, w[i])
        if w[i] > 30:
            gammapsd = np.append(gammapsd, psd[i])
            gammaw = np.append(gammaw, w[i])
    # averaged power for each frequency bands [delta_p,theta_p,alpha_p,beta_p,gamma_p]
    delta_p = np.mean(deltapsd)
    theta_p = np.mean(thetapsd)
    alpha_p = np.mean(alphapsd)
    beta_p = np.mean(betapsd)
    gamma_p = np.mean(gammapsd)
    overall_p = np.mean(psd)
    # relative power for each frequency bands
    r_delta_p = delta_p/overall_p
    r_theta_p = theta_p/overall_p
    r_alpha_p = alpha_p/overall_p
    r_beta_p = beta_p/overall_p
    r_gamma_p = gamma_p/overall_p
    # power ratios (δ/α), (δ/β),(δ/θ),(θ/α),(θ/β),(α/β),(δ+θ)/(α+β)
    r1 = delta_p/alpha_p
    r2 = delta_p/beta_p
    r3 = delta_p/theta_p
    r4 = theta_p/alpha_p
    r5 = theta_p/beta_p
    r6 = alpha_p/beta_p
    r7 = (delta_p+theta_p)/(alpha_p+beta_p)
    # SEF95
    cumulated_p = 0
    th = sum(psd)*0.95
    for i,f in enumerate(w):
        cumulated_p = cumulated_p + psd[i]
        if cumulated_p > th:
            sef95 = f
            break
# time domain
    # 75th percentile
    per_75_1 = np.quantile(eeg1, .75)
    per_75_2 = np.quantile(eeg2, .75)
    # zero crossing rate
    window_s = 0
    window_e = 201
    ZCRs1 = []
    ZCRs2 = []
    for i in range(213):
        zero1 = np.mean(eeg1[window_s:window_e])
        zcr1 = 0
        zero2 = np.mean(eeg2[window_s:window_e])
        zcr2 = 0
        for j in range(window_s,window_e):
            if (eeg1[j]-zero1)*(eeg1[j+1]-zero1)<0:
                zcr1 += 1
            if (eeg2[j]-zero2)*(eeg2[j+1]-zero2)<0:
                zcr2 += 1
        ZCRs1.append(zcr1)
        ZCRs2.append(zcr2)
    [zmean1,zstd1,zmaxv1,zminv1] = StatFeature(ZCRs1)
    [zmean2,zstd2,zmaxv2,zminv2] = StatFeature(ZCRs2)
    return np.array([delta_p,theta_p,alpha_p,beta_p,gamma_p,r_delta_p,overall_p,r_theta_p,r_alpha_p,r_beta_p,r_gamma_p,r1,r2,r3,r4,r5,r6,r7,sef95,per_75_1,per_75_2,zmean1,zstd1,zmaxv1,zminv1,zmean2,zstd2,zmaxv2,zminv2])

In [ ]:
a = np.array([1,2,3,4,5,6,7])
a[1:3]

In [ ]:
np.quantile([1,2,3,4,4,4],0.25)

In [ ]:
bio_signal=np.concatenate(([x_1[0][0]],[x_1[1][0]])).transpose() 
bio_signal.shape

In [ ]:
[_, theta, alpha_low,alpha_high,beta, gamma]=eeg.get_power_features(signal=bio_signal, sampling_rate=128,size=0.5)
gamma.shape

In [ ]:
x_1_features=[]
x_2_features=[]
x_3_features=[]
x_2_test_features=[]
x_1_test_features=[]


In [ ]:
subjects = [x_1,x_2,x_3,x_1_test,x_2_test]
subjects_f = [x_1_features,x_2_features,x_3_features,x_1_test_features,x_2_test_features]

for j,s in enumerate(subjects):
    for i in range(21600):
        if(i%200==0):
                print("Processing the ",i/200," batch of 200 signals...")
        raw_signal = np.concatenate(([s[0][i]],[s[1][i]]))
        feature0 = ExtractFeatures(np.array(s[0][i]),np.array(s[1][i]),np.array(s[2][i]))
        subjects_f[j].append(feature0)

In [ ]:

for i in range(21600):
    if(i%200==0):
            print("Processing the ",i/200," batch of 200 signals...")
    raw_signal = np.concatenate(([x_2[0][i]],[x_2[1][i]]))
    feature0 = ExtractFeatures(np.array(x_2[0][i]),np.array(x_2[1][i]),np.array(x_2[2][i]))
    x_2_features.append(feature0)

In [ ]:

for i in range(21600):
    if(i%200==0):
            print("Processing the ",i/200," batch of 200 signals...")
#     raw_signal = np.concatenate(([x_3[0][i]],[x_3[1][i]]))
    feature0 = ExtractFeatures(np.array(x_3[0][i]),np.array(x_3[1][i]),np.array(x_3[2][i]))
    x_3_features.append(feature0)

In [ ]:

for i in range(21600):
    if(i%200==0):
            print("Processing the ",i/200," batch of 200 signals...")
#     raw_signal = np.concatenate(([x_1_test[0][i]],[x_1_test[1][i]]))
    feature0 = ExtractFeatures(np.array(x_1_test[0][i]),np.array(x_1_test[1][i]),np.array(x_1_test[2][i]))
    x_1_test_features.append(feature0)

In [ ]:

for i in range(21600):
    if(i%200==0):
            print("Processing the ",i/200," batch of 200 signals...")
#     raw_signal = np.concatenate(([x_2_test[0][i]],[x_2_test[1][i]]))
    feature0 = ExtractFeatures(np.array(x_2_test[0][i]),np.array(x_2_test[1][i]),np.array(x_2_test[2][i]))
    x_2_test_features.append(feature0)

In [ ]:
def write_to_file_features(feature_M):
    pd.DataFrame(eval(feature_M)).to_csv("%s.csv"%(feature_M),index=False)

In [ ]:
write_to_file_features("x_1_features")
write_to_file_features("x_2_features")
write_to_file_features("x_3_features")
write_to_file_features("x_1_test_features")
write_to_file_features("x_2_test_features")

In [ ]:
x_1_features = np.array(x_1_features)
x_2_features = np.array(x_2_features)
x_3_features = np.array(x_3_features)
x_1_test_features = np.array(x_1_test_features)
x_2_test_features = np.array(x_2_test_features)

In [ ]:
# x_1_features = x_1_features[:,0:79]
# x_2_features = x_2_features[:,0:79]
# x_3_features = x_3_features[:,0:79]
# x_1_test_features = x_1_test_features[:,0:79]
# x_2_test_features = x_2_test_features[:,0:79]

In [ ]:
subjects = [x_1,x_2,x_3,x_1_test,x_2_test]
subjects_f = [x_1_features,x_2_features,x_3_features,x_1_test_features,x_2_test_features]
add_to_all_s = []
for s in subjects:
    listf = []
    for i in range(21600):
        if(i%200==0):
                print("Processing the ",i/200," batch of 200 signals...")
        raw_signal = np.concatenate(([s[0][i]],[s[1][i]]))
        feature_add = AddExtractFeatures(np.array(s[0][i]),np.array(s[1][i]),np.array(s[2][i]))
        listf.append(feature_add)
    add_to_all_s.append(listf)
x_1_features_a = np.concatenate((x_1_features,add_to_all_s[0]),axis=1)
x_2_features_a = np.concatenate((x_2_features,add_to_all_s[1]),axis=1)
x_3_features_a = np.concatenate((x_3_features,add_to_all_s[2]),axis=1)
x_1_test_features_a = np.concatenate((x_1_test_features,add_to_all_s[3]),axis=1)
x_2_test_features_a = np.concatenate((x_2_test_features,add_to_all_s[4]),axis=1)

In [ ]:
subjects = [x_1,x_2,x_3,x_1_test,x_2_test]

add_to_all_s = []
for s in subjects:
    listf = []
    for i in range(21600):
        if(i%200==0):
                print("Processing the ",i/200," batch of 200 signals...")
        raw_signal = np.concatenate(([s[0][i]],[s[1][i]]))
        feature_add = AddExtractFeatures(np.array(s[0][i]),np.array(s[1][i]),np.array(s[2][i]))
        listf.append(feature_add)
    add_to_all_s.append(listf)

Processing the  2.0  batch of 200 signals...
Processing the  3.0  batch of 200 signals...
Processing the  4.0  batch of 200 signals...
Processing the  5.0  batch of 200 signals...
Processing the  6.0  batch of 200 signals...
Processing the  7.0  batch of 200 signals...
Processing the  8.0  batch of 200 signals...
Processing the  9.0  batch of 200 signals...
Processing the  10.0  batch of 200 signals...
Processing the  11.0  batch of 200 signals...
Processing the  12.0  batch of 200 signals...
Processing the  13.0  batch of 200 signals...
Processing the  14.0  batch of 200 signals...
Processing the  15.0  batch of 200 signals...
Processing the  16.0  batch of 200 signals...
Processing the  17.0  batch of 200 signals...
Processing the  18.0  batch of 200 signals...
Processing the  19.0  batch of 200 signals...
Processing the  20.0  batch of 200 signals...
Processing the  21.0  batch of 200 signals...
Processing the  22.0  batch of 200 signals...
Processing the  23.0  batch of 200 signals

In [ ]:
x_1_features_a.shape

In [ ]:
write_to_file_features("add_to_all_s[0]")
write_to_file_features("add_to_all_s[1]")
write_to_file_features("add_to_all_s[2]")
write_to_file_features("add_to_all_s[3]")
write_to_file_features("add_to_all_s[4]")

In [ ]:
write_to_file_features("x_1_features_a")
write_to_file_features("x_2_features_a")
write_to_file_features("x_3_features_a")
write_to_file_features("x_1_test_features_a")
write_to_file_features("x_2_test_features_a")